# Project 6 - Exceptions

#### Description

Suppose we have a Widget online sales application and we are writing the backend for it. We want a base `WidgetException` class that we will use as the base class for all our custom exceptions we raise from our Widget application.

Furthermore we have determined that we will need the following categories of exceptions:

```
1. Supplier exceptions
    a. Not manufactured anymore
    b. Production delayed
    c. Shipping delayed
    
2. Checkout exceptions
    a. Inventory type exceptions
        - out of stock
    b. Pricing exceptions
        - invalid coupon code
        - cannot stack coupons
```

Write an exception class hierarchy to capture this. In addition, we would like to implement the following functionality:
* implement separate internal error message and user error message
* implement an http status code associated to each exception type (keep it simple, use a 500 (server error) error for everything except invalid coupon code, and cannot stack coupons, these can be 400 (bad request)
* implement a logging function that can be called to log the exception details, time it occurred, etc.
* implement a function that can be called to produce a json string containing the exception details you want to display to your user (include http status code (e.g. 400), the user error message, etc)

##### Bonus

Log the traceback too - you'll have to do a bit of research for that! 

I'm going to use the `TracebackException` class in the `traceback` module:

https://docs.python.org/3/library/traceback.html#tracebackexception-objects

In particular, look at the class method `from_exception` (and remember that inside your exception class, the exception will be `self`!) and the `format` instance method. That method returns a generator, so you'll need to `list` it to print out everything in that traceback.

#### Solution

Let's first define our base class:

Note: To get the traceback string that python (or jupyter) prints out for us, we need to use the `traceback` module. Since the `traceback.TracebackException.from_exception` expects an exception instance, we need to pass `self`. This function returns a generator which we'll need to consume inside our `log_exception` method.

In [21]:
from http import HTTPStatus
from datetime import datetime
import traceback
import json

In [22]:
class WidgetException(Exception):
    message = 'Generic Widget exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
    def __init__(self, *args, customer_message=None):
        super().__init__(*args)
        if args:
            self.message = args[0]
        self.customer_message = customer_message if customer_message is not None else self.message
        
    @property
    def traceback(self):
        return traceback.TracebackException.from_exception(self).format()
    
    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "message": self.message,
            "args": self.args[1:],
            "traceback": list(self.traceback)
        }
        print(f'EXCEPTION: {datetime.utcnow().isoformat()}: {exception}')
        
    def to_json(self):
        response = {
            'code': self.http_status.value,
            'message': '{}: {}'.format(self.http_status.phrase, self.customer_message),
            'category': type(self).__name__,
            'time_utc': datetime.utcnow().isoformat()            
        }
        return json.dumps(response)

Now we can define our exception sub types, including the http status for each:

In [23]:
class SupplierException(WidgetException):
    message = 'Supplier exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR

class NotManufacturedException(SupplierException):
    message = 'Widget is no longer manufactured by supplier.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class ProductionDelayedException(SupplierException):
    message = 'Widget production has been delayed by supplier.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class ShippingDelayedException(SupplierException):
    message = 'Widget shipping has been delayed by supplier.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class CheckoutException(WidgetException):
    message = 'Checkout exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class InventoryException(CheckoutException):
    message = 'Checkout inventory exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class OutOfStockException(InventoryException):
    message = 'Inventory out of stock'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class PricingException(CheckoutException):
    message = 'Checkout pricing exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class InvalidCouponCodeException(PricingException):
    message = 'Invalid checkout coupon code.'
    http_status = HTTPStatus.BAD_REQUEST
    
class CannotStackCouponException(PricingException):
    message = 'Cannot stack checkout coupon codes.'
    http_status = HTTPStatus.BAD_REQUEST

Let's try out an example:

In [24]:
e = InvalidCouponCodeException('User tried to pull a fast one on us', customer_message='Sorry. This coupon has expired.')

In [25]:
e.log_exception()

EXCEPTION: 2024-10-25T11:19:54.760052: {'type': 'InvalidCouponCodeException', 'message': 'User tried to pull a fast one on us', 'args': (), 'traceback': ['InvalidCouponCodeException: User tried to pull a fast one on us\n']}


In [26]:
e.to_json()

'{"code": 400, "message": "Bad Request: Sorry. This coupon has expired.", "category": "InvalidCouponCodeException", "time_utc": "2024-10-25T11:19:54.952985"}'

Note:
- how we pass the user a message different to what we log.
- our traceback was empty above (the exception is present, but there is no call stack) - because we did not actually raise the exception!

Let's actually raise the exception now:

In [27]:
try:
    raise ValueError
except ValueError:
    try:
        raise InvalidCouponCodeException(
            'User tried to use an old coupon', customer_message='Sorry. This coupon has expired.'
        )
    except InvalidCouponCodeException as ex:
        ex.log_exception()
        print('------------')
        print(ex.to_json())
        print('------------')
        print(''.join(ex.traceback))

EXCEPTION: 2024-10-25T11:22:28.142061: {'type': 'InvalidCouponCodeException', 'message': 'User tried to use an old coupon', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "C:\\Users\\nasiq\\AppData\\Local\\Temp\\ipykernel_12224\\3487775253.py", line 2, in <module>\n    raise ValueError\n', 'ValueError\n', '\nDuring handling of the above exception, another exception occurred:\n\n', 'Traceback (most recent call last):\n', '  File "C:\\Users\\nasiq\\AppData\\Local\\Temp\\ipykernel_12224\\3487775253.py", line 5, in <module>\n    raise InvalidCouponCodeException(\n', 'InvalidCouponCodeException: User tried to use an old coupon\n']}
------------
{"code": 400, "message": "Bad Request: Sorry. This coupon has expired.", "category": "InvalidCouponCodeException", "time_utc": "2024-10-25T11:22:28.142061"}
------------
Traceback (most recent call last):
  File "C:\Users\nasiq\AppData\Local\Temp\ipykernel_12224\3487775253.py", line 2, in <module>
    raise ValueError
Value